# SR Merged Product 메타데이터 생성

이 노트북은 `sr_merged_product` 테이블에 대한 통계 정보와 설명을 생성합니다.

## 작업 내용:
1. PostgreSQL 연결 및 테이블 스키마 확인
2. 테이블 통계 정보 수집
3. 컬럼별 통계 분석
4. Azure OpenAI를 활용한 설명 생성
5. 메타데이터 저장

In [4]:
# 1. 필요한 라이브러리 임포트
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# .env 파일 로드
load_dotenv()

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [5]:
# 2. PostgreSQL 연결 설정
def get_db_connection():
    """PostgreSQL 데이터베이스 연결"""
    try:
        conn = psycopg2.connect(
            host=os.getenv('PG_HOST'),
            port=os.getenv('PG_PORT'),
            database=os.getenv('PG_DATABASE'),
            user=os.getenv('PG_USER'),
            password=os.getenv('PG_PASSWORD')
        )
        print(f"✅ PostgreSQL 연결 성공: {os.getenv('PG_HOST')}")
        return conn
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        return None

# 연결 테스트
conn = get_db_connection()
if conn:
    conn.close()

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com


In [6]:
# 3. Azure OpenAI 클라이언트 설정
def get_openai_client():
    """Azure OpenAI 클라이언트 생성"""
    try:
        # 환경 변수 확인
        endpoint = os.getenv('ENDPOINT_URL')
        api_key = os.getenv('AZURE_OPENAI_API_KEY')
        api_version = os.getenv('AZURE_API_VERSION', '2024-02-01')  # 기본값 제공
        
        if not endpoint:
            print("❌ ENDPOINT_URL 환경 변수가 설정되지 않았습니다.")
            return None
        
        if not api_key:
            print("❌ AZURE_OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
            return None
        
        # API 버전이 없으면 기본값 사용
        if not api_version:
            api_version = '2024-02-01'
            print(f"⚠️ AZURE_API_VERSION이 설정되지 않아 기본값 사용: {api_version}")
        
        print(f"📋 Azure OpenAI 설정:")
        print(f"  - Endpoint: {endpoint[:50]}...")
        print(f"  - API Version: {api_version}")
        print(f"  - Deployment: {os.getenv('DEPLOYMENT_NAME')}")
        
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
        )
        print(f"✅ Azure OpenAI 클라이언트 생성 성공")
        return client
    except Exception as e:
        print(f"❌ Azure OpenAI 클라이언트 생성 실패: {e}")
        return None

# 클라이언트 테스트
openai_client = get_openai_client()

# 간단한 테스트 호출
if openai_client:
    try:
        print("\n🧪 API 연결 테스트...")
        test_response = openai_client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Say 'Hello' in one word."}
            ],
            max_tokens=10
        )
        if test_response and test_response.choices:
            print(f"✅ API 테스트 성공: '{test_response.choices[0].message.content}'")
        else:
            print("⚠️ API 응답이 비어있습니다.")
    except Exception as e:
        print(f"❌ API 테스트 실패: {e}")
        print(f"   상세 에러: {type(e).__name__}")
        if hasattr(e, '__dict__'):
            print(f"   에러 속성: {e.__dict__}")

📋 Azure OpenAI 설정:
  - Endpoint: https://kt-azure-openai-dev-donghunseo.openai.azur...
  - API Version: 2024-02-01
  - Deployment: gpt-4.1
✅ Azure OpenAI 클라이언트 생성 성공

🧪 API 연결 테스트...
✅ API 테스트 성공: 'Hello'


In [7]:
# 4. 테이블 스키마 정보 조회
def get_table_schema(table_name='test'):
    """테이블 스키마 정보 조회 (코멘트 포함)"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # PostgreSQL에서 컬럼 정보와 코멘트를 함께 조회
        query = """
        SELECT 
            c.column_name,
            c.data_type,
            c.character_maximum_length,
            c.numeric_precision,
            c.numeric_scale,
            c.is_nullable,
            c.column_default,
            pgd.description as column_comment
        FROM information_schema.columns c
        LEFT JOIN pg_catalog.pg_statio_all_tables as st
            ON c.table_schema = st.schemaname 
            AND c.table_name = st.relname
        LEFT JOIN pg_catalog.pg_description pgd 
            ON pgd.objoid = st.relid 
            AND pgd.objsubid = c.ordinal_position
        WHERE c.table_schema = 'public' 
        AND c.table_name = %s
        ORDER BY c.ordinal_position;
        """
        
        df_schema = pd.read_sql_query(query, conn, params=(table_name,))
        print(f"✅ 테이블 '{table_name}' 스키마 조회 성공")
        print(f"   - 컬럼 수: {len(df_schema)}")
        
        # 코멘트가 있는 컬럼 수 확인
        comment_count = df_schema['column_comment'].notna().sum()
        print(f"   - 코멘트가 있는 컬럼: {comment_count}개")
        
        return df_schema
    
    except Exception as e:
        print(f"❌ 테이블 스키마 조회 실패: {e}")
        return None
    
    finally:
        conn.close()

# 스키마 조회
df_schema = get_table_schema(table_name="kt_merged_product_20250929")
if df_schema is not None:
    print("\n테이블 스키마 정보:")
    display(df_schema.head(10))

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 테이블 'kt_merged_product_20250929' 스키마 조회 성공
   - 컬럼 수: 34
   - 코멘트가 있는 컬럼: 34개

테이블 스키마 정보:


,column_name,data_type,character_maximum_length,numeric_precision,numeric_scale,is_nullable,column_default,column_comment
0,display_category_major,character varying,1000.0,NaN,NaN,YES,None,"웹사이트에 노출되는 제품의 대분류 카테고리명입니다. (예: 모바일, TV & 오디오..."
1,display_category_middle,character varying,1000.0,NaN,NaN,YES,None,"웹사이트에 노출되는 제품의 중분류 카테고리명입니다. (예: 스마트폰, QLED, 비..."
2,display_category_minor,character varying,1000.0,NaN,NaN,YES,None,"웹사이트에 노출되는 제품의 소분류 카테고리명입니다. (예: Galaxy S, Neo..."
3,product_category_major,character varying,1000.0,NaN,NaN,YES,None,내부 시스템에서 관리하는 제품의 대분류 카테고리 코드 또는 이름입니다.
4,product_category_middle,character varying,1000.0,NaN,NaN,YES,None,내부 시스템에서 관리하는 제품의 중분류 카테고리 코드 또는 이름입니다.
5,product_category_minor,character varying,1000.0,NaN,NaN,YES,None,내부 시스템에서 관리하는 제품의 소분류 카테고리 코드 또는 이름입니다.
6,model_name,character varying,1000.0,NaN,NaN,YES,None,제품의 공식 모델명입니다. 사용자가 일반적으로 인지하는 이름입니다. (예: Gala...
7,model_code,character varying,1000.0,NaN,NaN,YES,None,"제품의 세부 사양(색상, 용량 등)을 포함하는 고유 모델 코드입니다. (예: SM-..."
8,product_id,character varying,1000.0,NaN,NaN,YES,None,데이터베이스에서 각 제품을 식별하기 위한 고유 ID입니다.
9,product_name,character varying,1000.0,NaN,NaN,YES,None,웹사이트에 최종적으로 표시되는 제품명입니다. (예: 갤럭시 S24 Ultra 자급제...


In [8]:
# 5. 테이블 기본 통계 정보 수집
def get_table_statistics(table_name='test'):
    """테이블 기본 통계 정보 수집"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        statistics = {}
        cursor = conn.cursor()
        
        # 테이블 행 수
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        statistics['total_rows'] = cursor.fetchone()[0]
        
        # 테이블 크기
        cursor.execute(f"""
            SELECT pg_size_pretty(pg_relation_size('{table_name}')) as table_size,
                   pg_size_pretty(pg_total_relation_size('{table_name}')) as total_size
        """)
        size_info = cursor.fetchone()
        if size_info:
            statistics['table_size'] = size_info[0]
            statistics['total_size'] = size_info[1]
        
        # 테이블 통계 정보 (pg_stat_user_tables의 실제 컬럼명 사용)
        cursor.execute(f"""
            SELECT schemaname, relname, n_tup_ins, n_tup_upd, n_tup_del,
                   last_vacuum, last_autovacuum, last_analyze, last_autoanalyze
            FROM pg_stat_user_tables
            WHERE relname = '{table_name}'
        """)
        stat_info = cursor.fetchone()
        if stat_info:
            statistics['stats'] = {
                'schema': stat_info[0],
                'table': stat_info[1],
                'inserts': stat_info[2],
                'updates': stat_info[3],
                'deletes': stat_info[4],
                'last_vacuum': str(stat_info[5]) if stat_info[5] else None,
                'last_autovacuum': str(stat_info[6]) if stat_info[6] else None,
                'last_analyze': str(stat_info[7]) if stat_info[7] else None,
                'last_autoanalyze': str(stat_info[8]) if stat_info[8] else None
            }
        
        print(f"✅ 테이블 '{table_name}' 통계 정보 수집 완료")
        return statistics
    
    except Exception as e:
        print(f"❌ 테이블 통계 정보 수집 실패: {e}")
        # 에러 발생 시에도 기본 정보는 반환
        try:
            cursor = conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            return {'total_rows': count}
        except:
            return None
    
    finally:
        if 'cursor' in locals():
            cursor.close()
        conn.close()

# 통계 정보 수집
table_stats = get_table_statistics(table_name="kt_merged_product_20250929")
if table_stats:
    print("\n테이블 통계 정보:")
    print(json.dumps(table_stats, indent=2, default=str))

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 테이블 'kt_merged_product_20250929' 통계 정보 수집 완료

테이블 통계 정보:
{
  "total_rows": 2774,
  "table_size": "2640 kB",
  "total_size": "6152 kB",
  "stats": {
    "schema": "public",
    "table": "kt_merged_product_20250929",
    "inserts": 2774,
    "updates": 0,
    "deletes": 0,
    "last_vacuum": null,
    "last_autovacuum": "2025-09-29 11:41:29.701508+00:00",
    "last_analyze": null,
    "last_autoanalyze": "2025-09-29 11:41:31.693453+00:00"
  }
}


In [9]:
# 6. 컬럼별 상세 통계 분석
def get_column_statistics(table_name='test', sample_size=10000):
    """컬럼별 상세 통계 정보 수집"""
    conn = get_db_connection()
    if not conn:
        return None

    try:
        # 먼저 테이블에 데이터가 있는지 확인
        cursor = conn.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        total_rows = cursor.fetchone()[0]

        if total_rows == 0:
            print(f"⚠️ 테이블 '{table_name}'에 데이터가 없습니다.")
            return None

        # 샘플 크기 조정 (전체 행 수보다 크면 전체 행 수로 조정)
        actual_sample_size = min(sample_size, total_rows)

        # 샘플 데이터 로드
        query = f"SELECT * FROM {table_name} LIMIT {actual_sample_size}"
        df = pd.read_sql_query(query, conn)

        column_stats = []

        for col in df.columns:
            stats = {
                'column_name': col,
                'data_type': str(df[col].dtype),
                'non_null_count': int(df[col].notna().sum()),
                'null_count': int(df[col].isna().sum()),
                'null_ratio': f"{df[col].isna().mean() * 100:.2f}%",
                # 기본적으로 모든 통계 값을 None으로 초기화
                'min': None,
                'max': None,
                'mean': None,
                'median': None,
                'std': None
            }
            
            # 스키마 정보에서 코멘트 추가
            if df_schema is not None and 'column_comment' in df_schema.columns:
                schema_row = df_schema[df_schema['column_name'] == col]
                if not schema_row.empty:
                    stats['column_comment'] = schema_row.iloc[0]['column_comment']
                else:
                    stats['column_comment'] = None
            else:
                stats['column_comment'] = None

            # unique_count 계산 시 에러 처리
            try:
                # JSON/JSONB 컬럼이나 복잡한 객체가 포함된 경우를 처리
                if df[col].dtype == 'object':
                    # 문자열로 변환하여 고유값 계산
                    unique_count = df[col].astype(str).nunique()
                else:
                    unique_count = df[col].nunique()

                stats['unique_count'] = int(unique_count)
                stats['unique_ratio'] = f"{unique_count / len(df) * 100:.2f}%"
            except Exception as e:
                # 에러 발생 시 기본값 설정
                stats['unique_count'] = None
                stats['unique_ratio'] = None
                print(f"  ⚠️ 컬럼 '{col}' unique_count 계산 실패: {e}")

            # 수치형 데이터 통계 (숫자형 컬럼에만 적용)
            if pd.api.types.is_numeric_dtype(df[col]):
                # null이 아닌 값만 추출
                non_null_values = df[col].dropna()
                if len(non_null_values) > 0:
                    try:
                        stats['min'] = float(non_null_values.min())
                    except:
                        stats['min'] = None

                    try:
                        stats['max'] = float(non_null_values.max())
                    except:
                        stats['max'] = None

                    try:
                        stats['mean'] = float(non_null_values.mean())
                    except:
                        stats['mean'] = None

                    try:
                        stats['median'] = float(non_null_values.median())
                    except:
                        stats['median'] = None

                    try:
                        # std는 샘플이 2개 이상일 때만 계산 가능
                        if len(non_null_values) > 1:
                            stats['std'] = float(non_null_values.std())
                        else:
                            stats['std'] = None
                    except:
                        stats['std'] = None
                else:
                    # non_null_values가 없으면 모든 통계 값은 이미 None으로 설정됨
                    pass

            # 문자열 및 객체 데이터 통계
            elif df[col].dtype == 'object':
                non_null_values = df[col].dropna()
                if len(non_null_values) > 0:
                    try:
                        # 문자열로 변환하여 길이 계산
                        str_values = non_null_values.astype(str)
                        str_lengths = str_values.str.len()

                        stats.update({
                            'min_length': int(str_lengths.min()) if len(str_lengths) > 0 else None,
                            'max_length': int(str_lengths.max()) if len(str_lengths) > 0 else None,
                            'avg_length': float(str_lengths.mean()) if len(str_lengths) > 0 else None
                        })

                        # most_common 계산 시 에러 처리
                        try:
                            # 복잡한 객체는 문자열로 변환하여 카운트
                            value_counts = df[col].astype(str).value_counts().head(3)
                            stats['most_common'] = value_counts.to_dict()
                        except:
                            stats['most_common'] = {}

                    except Exception as e:
                        print(f"  ⚠️ 컬럼 '{col}' 문자열 통계 계산 실패: {e}")
                        stats.update({
                            'min_length': None,
                            'max_length': None,
                            'avg_length': None,
                            'most_common': {}
                        })
                else:
                    stats.update({
                        'min_length': None,
                        'max_length': None,
                        'avg_length': None,
                        'most_common': {}
                    })

            # 날짜형 데이터 통계
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                non_null_values = df[col].dropna()
                if len(non_null_values) > 0:
                    stats.update({
                        'min_date': str(non_null_values.min()),
                        'max_date': str(non_null_values.max()),
                        'date_range': str(non_null_values.max() - non_null_values.min())
                    })
                else:
                    stats.update({
                        'min_date': None,
                        'max_date': None,
                        'date_range': None
                    })

            column_stats.append(stats)

        print(f"✅ 컬럼별 통계 분석 완료 (샘플 크기: {len(df)}행)")
        return pd.DataFrame(column_stats)

    except Exception as e:
        print(f"❌ 컬럼별 통계 분석 실패: {e}")
        import traceback
        traceback.print_exc()
        return None

    finally:
        if 'cursor' in locals():
            cursor.close()
        conn.close()

# 컬럼 통계 수집
df_column_stats = get_column_statistics(table_name="kt_merged_product_20250929")
if df_column_stats is not None:
    print(f"\n총 {len(df_column_stats)}개 컬럼 분석 완료")
    print("\n컬럼별 통계 정보 (처음 20개):")
    display(df_column_stats.head(20))

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 컬럼별 통계 분석 완료 (샘플 크기: 2774행)

총 34개 컬럼 분석 완료

컬럼별 통계 정보 (처음 20개):


,column_name,data_type,non_null_count,null_count,null_ratio,min,max,mean,median,std,column_comment,unique_count,unique_ratio,min_length,max_length,avg_length,most_common
0,display_category_major,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,"웹사이트에 노출되는 제품의 대분류 카테고리명입니다. (예: 모바일, TV & 오디오...",8,0.29%,2.0,8.0,3.782624,"{'모바일': 813, '리빙가전': 592, 'TV': 430}"
1,display_category_middle,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,"웹사이트에 노출되는 제품의 중분류 카테고리명입니다. (예: 스마트폰, QLED, 비...",41,1.48%,2.0,18.0,6.776136,"{'갤럭시 액세서리': 455, 'TV': 347, '시스템에어컨': 192}"
2,display_category_minor,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,"웹사이트에 노출되는 제품의 소분류 카테고리명입니다. (예: Galaxy S, Neo...",183,6.60%,2.0,31.0,8.112833,"{'케이스': 269, '갤럭시 S': 109, '상업용': 102}"
3,product_category_major,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,내부 시스템에서 관리하는 제품의 대분류 카테고리 코드 또는 이름입니다.,20,0.72%,2.0,34.0,10.736842,"{'HHP': 804, 'Flat Panel Displayer': 430, 'AIR..."
4,product_category_middle,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,내부 시스템에서 관리하는 제품의 중분류 카테고리 코드 또는 이름입니다.,80,2.88%,2.0,32.0,13.322999,"{'MOBILE-VPS PRODUCT': 354, 'QLED TV': 263, 'N..."
5,product_category_minor,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,내부 시스템에서 관리하는 제품의 소분류 카테고리 코드 또는 이름입니다.,200,7.21%,2.0,29.0,11.080750,"{'Case': 241, 'Air Conditioner': 192, 'Cartrid..."
6,model_name,object,2697,77,2.78%,NaN,NaN,NaN,NaN,NaN,제품의 공식 모델명입니다. 사용자가 일반적으로 인지하는 이름입니다. (예: Gala...,2120,76.42%,1.0,18.0,10.537264,"{'None': 77, 'SM-S938N': 25, 'SM-F741N': 22}"
7,model_code,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,"제품의 세부 사양(색상, 용량 등)을 포함하는 고유 모델 코드입니다. (예: SM-...",2774,100.00%,5.0,18.0,12.701514,"{'JBLQUANTUM300BLK': 1, 'SM-X526NZAEKOO': 1, '..."
8,product_id,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,데이터베이스에서 각 제품을 식별하기 위한 고유 ID입니다.,2774,100.00%,10.0,10.0,10.000000,"{'G000190421': 1, 'G000433988': 1, 'G000433974..."
9,product_name,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,웹사이트에 최종적으로 표시되는 제품명입니다. (예: 갤럭시 S24 Ultra 자급제...,1681,60.60%,3.0,120.0,28.925018,"{'갤럭시 링': 33, '갤럭시 S25 울트라 자급제': 12, 'Bespoke ..."


In [10]:
display(df_column_stats.tail(20))

,column_name,data_type,non_null_count,null_count,null_ratio,min,max,mean,median,std,column_comment,unique_count,unique_ratio,min_length,max_length,avg_length,most_common
14,is_galaxy_club_eligible,object,46,2728,98.34%,NaN,NaN,NaN,NaN,NaN,갤럭시 클럽 가입 또는 혜택 적용 가능 여부를 나타냅니다. (True/False),2,0.07%,1.0,1.0,1.000000,"{'None': 2728, 'N': 46}"
15,is_installment_payment_available,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,할부 결제 가능 여부를 나타냅니다. (True/False),2,0.07%,1.0,1.0,1.000000,"{'Y': 2208, 'N': 566}"
16,product_detail_url,object,2774,0,0.00%,NaN,NaN,NaN,NaN,NaN,제품의 상세 페이지로 연결되는 전체 URL 주소입니다.,2774,100.00%,53.0,138.0,83.071017,{'https://www.samsung.com/sec/harman-life-styl...
17,channel_code,object,0,2774,100.00%,NaN,NaN,NaN,NaN,NaN,"판매 채널을 구분하는 코드입니다. (예: 온라인, B2B, 특정 프로모션)",1,0.04%,NaN,NaN,NaN,{}
18,unique_selling_point,object,2666,108,3.89%,NaN,NaN,NaN,NaN,NaN,"제품의 핵심 특장점을 요약한 문구 모음입니다. (예: 'Galaxy AI 탑재', ...",880,31.72%,2.0,160.0,76.400975,"{'None': 108, '{""다양한 패널로 완성하는 나만의 냉장고"",""인테리어에 ..."
19,review_count,float64,2706,68,2.45%,0.0,8464.0,4.424183e+02,20.00,1.398079e+03,해당 제품에 달린 리뷰의 총 개수입니다.,246,8.87%,NaN,NaN,NaN,NaN
20,review_rating_score,float64,2706,68,2.45%,0.0,5.0,3.723899e+00,4.67,1.902013e+00,해당 제품의 평균 리뷰 별점입니다. (예: 4.8),122,4.40%,NaN,NaN,NaN,NaN
21,standard_price,float64,2774,0,0.00%,3300.0,52690000.0,1.890617e+06,676500.00,3.490187e+06,할인이 적용되지 않은 정상 판매가입니다.,968,34.90%,NaN,NaN,NaN,NaN
22,member_price,float64,2774,0,0.00%,3300.0,52690000.0,1.890277e+06,676500.00,3.490346e+06,삼성닷컴 회원에게만 적용되는 할인가입니다.,966,34.82%,NaN,NaN,NaN,NaN
23,benefit_price,float64,1714,1060,38.21%,3000.0,50700000.0,2.177257e+06,1390000.00,3.172367e+06,"즉시 할인, 쿠폰 등 모든 혜택이 적용된 최종 가격으로, '혜택가'에 해당합니다.",696,25.09%,NaN,NaN,NaN,NaN


In [11]:
# 7. Azure OpenAI를 활용한 컬럼 설명 생성
def generate_column_description(column_info, table_context='test'):
    """Azure OpenAI를 사용하여 컬럼 설명 생성"""
    
    if not openai_client:
        print("⚠️ OpenAI 클라이언트가 초기화되지 않았습니다.")
        return None
    
    # 데이터 타입 확인
    data_type = column_info.get('data_type', '')
    is_numeric = any(dtype in data_type.lower() for dtype in ['int', 'float', 'numeric', 'decimal', 'double'])
    is_string = 'object' in data_type.lower() or 'varchar' in data_type.lower() or 'text' in data_type.lower()
    is_date = 'date' in data_type.lower() or 'time' in data_type.lower()
    
    # 기본 정보
    base_info = f"""
    테이블명: {table_context}
    컬럼 정보:
    - 컬럼명: {column_info.get('column_name')}
    - 데이터 타입: {column_info.get('data_type')}
    - NULL 비율: {column_info.get('null_ratio', 'N/A')}
    - 고유값 개수: {column_info.get('unique_count', 'N/A')}"""
    
    # 기존 코멘트가 있으면 추가
    column_comment = column_info.get('column_comment')
    if column_comment and column_comment != 'None' and pd.notna(column_comment):
        base_info += f"\n    - 기존 설명: {column_comment}"
    
    # 데이터 타입별 추가 정보
    type_specific_info = ""
    
    if is_numeric:
        # 숫자형 데이터인 경우
        min_val = column_info.get('min')
        max_val = column_info.get('max')
        mean_val = column_info.get('mean')
        median_val = column_info.get('median')
        std_val = column_info.get('std')
        
        if min_val is not None or max_val is not None or mean_val is not None:
            type_specific_info += "\n    - 통계 정보:"
            if min_val is not None:
                type_specific_info += f"\n      - 최소값: {min_val}"
            if max_val is not None:
                type_specific_info += f"\n      - 최대값: {max_val}"
            if mean_val is not None:
                type_specific_info += f"\n      - 평균: {mean_val:.2f}"
            if median_val is not None:
                type_specific_info += f"\n      - 중앙값: {median_val:.2f}"
            if std_val is not None:
                type_specific_info += f"\n      - 표준편차: {std_val:.2f}"
    
    elif is_string:
        # 문자열 데이터인 경우
        min_length = column_info.get('min_length')
        max_length = column_info.get('max_length')
        avg_length = column_info.get('avg_length')
        most_common = column_info.get('most_common', {})
        
        if min_length is not None or max_length is not None or avg_length is not None:
            type_specific_info += "\n    - 문자열 길이 정보:"
            if min_length is not None:
                type_specific_info += f"\n      - 최소 길이: {min_length}"
            if max_length is not None:
                type_specific_info += f"\n      - 최대 길이: {max_length}"
            if avg_length is not None:
                type_specific_info += f"\n      - 평균 길이: {avg_length:.1f}"
        
        if most_common and len(most_common) > 0:
            type_specific_info += "\n    - 가장 빈번한 값:"
            for value, count in list(most_common.items())[:3]:
                # 긴 문자열은 잘라서 표시
                display_value = value if len(str(value)) <= 30 else str(value)[:30] + "..."
                type_specific_info += f"\n      - '{display_value}': {count}개"
    
    elif is_date:
        # 날짜형 데이터인 경우
        min_date = column_info.get('min_date')
        max_date = column_info.get('max_date')
        date_range = column_info.get('date_range')
        
        if min_date or max_date or date_range:
            type_specific_info += "\n    - 날짜 범위:"
            if min_date:
                type_specific_info += f"\n      - 최소 날짜: {min_date}"
            if max_date:
                type_specific_info += f"\n      - 최대 날짜: {max_date}"
            if date_range:
                type_specific_info += f"\n      - 날짜 범위: {date_range}"
    
    # 프롬프트 구성 - 기존 코멘트가 있는 경우와 없는 경우를 구분
    if column_comment and column_comment != 'None' and pd.notna(column_comment):
        prompt = f"""{base_info}{type_specific_info}
    
    기존 설명을 참고하여 다음 형식으로 개선된 설명을 생성해주세요:
    1. 짧은 설명 (한 줄, 20자 이내) - 기존 설명을 참고하여 더 명확하게
    2. 상세 설명 (2-3줄, 비즈니스 의미 포함) - 기존 설명을 확장하여
    3. 데이터 특성 (NULL 허용 여부, 값 범위 등)
    
    기존 설명이 충분히 명확하다면 그대로 사용하되, 통계 정보를 반영하여 보완해주세요.
    """
    else:
        prompt = f"""{base_info}{type_specific_info}
    
    다음 형식으로 응답해주세요:
    1. 짧은 설명 (한 줄, 20자 이내)
    2. 상세 설명 (2-3줄, 비즈니스 의미 포함)
    3. 데이터 특성 (NULL 허용 여부, 값 범위 등)
    """
    
    try:
        print(f"\n📝 API 요청 정보:")
        print(f"  - 모델: {os.getenv('DEPLOYMENT_NAME')}")
        print(f"  - 엔드포인트: {os.getenv('ENDPOINT_URL')[:50]}...")
        
        # 프롬프트 일부 출력 (디버깅용)
        print(f"  - 프롬프트 길이: {len(prompt)}자")
        print(f"  - 프롬프트 처음 200자:\n{prompt[:200]}...")
        
        response = openai_client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=[
                {"role": "system", "content": "당신은 데이터베이스 전문가입니다. 컬럼의 비즈니스 의미를 명확하게 설명해주세요. 기존 코멘트가 있다면 이를 참고하여 개선된 설명을 제공하세요."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=3000,
            temperature=1,
            # temperature 제거 (gpt-5-01 모델에서 지원 안 할 수 있음)
        )
        
        # 전체 응답 객체 확인
        print(f"\n📋 응답 객체 타입: {type(response)}")
        print(f"  - choices 개수: {len(response.choices) if response.choices else 0}")
        
        # 응답 확인
        if response and response.choices and len(response.choices) > 0:
            choice = response.choices[0]
            print(f"  - choice 객체: {choice}")
            print(f"  - finish_reason: {choice.finish_reason}")
            print(f"  - message 타입: {type(choice.message)}")
            
            # content 속성 확인
            if hasattr(choice.message, 'content'):
                content = choice.message.content
                print(f"  - content 타입: {type(content)}")
                print(f"  - content 값: '{content}'")
                
                if content:
                    print(f"✅ API 응답 수신 (길이: {len(content)}자)")
                    return content
                else:
                    print("⚠️ API 응답 content가 비어있습니다.")
                    # 빈 응답일 경우 기본 값 반환
                    if column_comment:
                        return f"1. {column_comment}\n2. {column_comment} 정보를 저장하는 컬럼입니다.\n3. NULL 허용, 문자열 타입"
                    else:
                        return f"1. {column_info.get('column_name')} 정보\n2. {column_info.get('column_name')} 관련 데이터를 저장합니다.\n3. {column_info.get('null_ratio')} NULL 비율"
            else:
                print("⚠️ message에 content 속성이 없습니다.")
                print(f"  - message 속성들: {dir(choice.message)}")
                return None
        else:
            print("⚠️ API 응답 형식이 예상과 다릅니다.")
            print(f"   응답 객체: {response}")
            return None
    
    except Exception as e:
        print(f"❌ OpenAI 설명 생성 실패: {e}")
        print(f"   에러 타입: {type(e).__name__}")
        if hasattr(e, 'response'):
            print(f"   응답 상태: {getattr(e.response, 'status_code', 'N/A')}")
            print(f"   응답 내용: {getattr(e.response, 'text', 'N/A')}")
        
        # 에러 발생 시 기본 설명 반환
        if column_comment:
            return f"1. {column_comment}\n2. {column_comment} 정보를 저장하는 컬럼입니다.\n3. NULL 허용 여부 확인 필요"
        else:
            return f"1. {column_info.get('column_name')} 컬럼\n2. 상세 설명 생성 실패\n3. 데이터 타입: {column_info.get('data_type')}"

# 테스트: 첫 번째 컬럼에 대한 설명 생성
if df_column_stats is not None and len(df_column_stats) > 0:
    print("=" * 60)
    print("테스트: 첫 번째 컬럼 설명 생성")
    print("=" * 60)
    
    test_column = df_column_stats.iloc[0].to_dict()
    print(f"\n테스트 컬럼 정보:")
    print(f"  - 컬럼명: {test_column.get('column_name')}")
    print(f"  - 데이터 타입: {test_column.get('data_type')}")
    print(f"  - 기존 코멘트: {test_column.get('column_comment')}")
    
    description = generate_column_description(test_column, table_context="kt_merged_product_20250929")
    
    print("\n" + "=" * 60)
    if description:
        print(f"생성된 설명:")
        print("-" * 50)
        print(description)
    else:
        print("⚠️ 설명 생성 실패 (None 반환)")
        
    # 추가 디버깅: 직접 API 호출 테스트
    print("\n" + "=" * 60)
    print("직접 API 테스트")
    print("=" * 60)
    
    if openai_client:
        try:
            print("간단한 메시지로 테스트 중...")
            test_response = openai_client.chat.completions.create(
                model=os.getenv('DEPLOYMENT_NAME'),
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": "Say 'Hello World' in Korean."}
                ],
                max_completion_tokens=50
            )
            
            print(f"테스트 응답 타입: {type(test_response)}")
            if test_response.choices:
                print(f"Choices 개수: {len(test_response.choices)}")
                print(f"첫 번째 choice: {test_response.choices[0]}")
                print(f"Message: {test_response.choices[0].message}")
                print(f"Content: '{test_response.choices[0].message.content}'")
            else:
                print("No choices in response")
                
        except Exception as test_e:
            print(f"테스트 실패: {test_e}")

테스트: 첫 번째 컬럼 설명 생성

테스트 컬럼 정보:
  - 컬럼명: display_category_major
  - 데이터 타입: object
  - 기존 코멘트: 웹사이트에 노출되는 제품의 대분류 카테고리명입니다. (예: 모바일, TV & 오디오, 가전)

📝 API 요청 정보:
  - 모델: gpt-4.1
  - 엔드포인트: https://kt-azure-openai-dev-donghunseo.openai.azur...
  - 프롬프트 길이: 582자
  - 프롬프트 처음 200자:

    테이블명: kt_merged_product_20250929
    컬럼 정보:
    - 컬럼명: display_category_major
    - 데이터 타입: object
    - NULL 비율: 0.00%
    - 고유값 개수: 8
    - 기존 설명: 웹사이트에 노출되는 제품의 대분류 카테고리명입니다. (예: 모바일, TV & 오디오...

📋 응답 객체 타입: <class 'openai.types.chat.chat_completion.ChatCompletion'>
  - choices 개수: 1
  - choice 객체: Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="1. 짧은 설명  \n제품의 대분류 카테고리명  \n\n2. 상세 설명  \n웹사이트에서 제품을 분류할 때 사용되는 대분류 카테고리명으로, '모바일', 'TV', '리빙가전' 등 8가지 주요 카테고리로 제품이 구분됩니다. 비즈니스적으로는 고객이 원하는 제품군을 빠르게 찾을 수 있도록 지원하며, 제품군별 분석 및 관리를 위한 기준이 됩니다.\n\n3. 데이터 특성  \nNULL 값은 없으며(0.00%), 총 8개의 고유 카테고리값만 존재합니다. 주요값은 '모바일', '리빙가전', 'TV' 등으로, 값의 문자열 길이는 최소 2자에서 최대 8자입니다.", refusal=Non

In [12]:
# 8. 모든 컬럼에 대한 메타데이터 생성 (배치 처리)
def generate_all_column_metadata(df_stats, batch_size=5):
    """모든 컬럼에 대한 메타데이터 생성"""
    
    if df_stats is None or len(df_stats) == 0:
        print("❌ 컬럼 통계 정보가 없습니다.")
        return None
    
    metadata_list = []
    total_columns = len(df_stats)
    
    print(f"총 {total_columns}개 컬럼에 대한 메타데이터 생성 시작...")
    
    for i in range(0, total_columns, batch_size):
        batch_end = min(i + batch_size, total_columns)
        print(f"\n배치 처리: {i+1}-{batch_end}/{total_columns}")
        
        for idx in range(i, batch_end):
            column_info = df_stats.iloc[idx].to_dict()
            column_name = column_info['column_name']
            
            print(f"  - {column_name} 처리 중...")
            
            # OpenAI 설명 생성
            description = generate_column_description(column_info, table_context="kt_merged_product_20250929")
            
            # 메타데이터 구성
            metadata = {
                'column_name': column_name,
                'data_type': column_info.get('data_type'),
                'null_ratio': column_info.get('null_ratio'),
                'unique_count': column_info.get('unique_count'),
                'description': description,
                'generated_at': datetime.now().isoformat()
            }
            
            # 수치형 데이터 추가 정보
            if 'min' in column_info:
                metadata.update({
                    'min': column_info.get('min'),
                    'max': column_info.get('max'),
                    'mean': column_info.get('mean'),
                    'median': column_info.get('median')
                })
            
            metadata_list.append(metadata)
            
            # API 호출 제한 방지를 위한 대기
            import time
            time.sleep(0.5)
    
    print(f"\n✅ 총 {len(metadata_list)}개 컬럼 메타데이터 생성 완료")
    return pd.DataFrame(metadata_list)

# 메타데이터 생성 (처음 10개 컬럼만)
df_metadata = generate_all_column_metadata(df_column_stats)
if df_metadata is not None:
    print("\n생성된 메타데이터:")
    display(df_metadata)

총 34개 컬럼에 대한 메타데이터 생성 시작...

배치 처리: 1-5/34
  - display_category_major 처리 중...

📝 API 요청 정보:
  - 모델: gpt-4.1
  - 엔드포인트: https://kt-azure-openai-dev-donghunseo.openai.azur...
  - 프롬프트 길이: 582자
  - 프롬프트 처음 200자:

    테이블명: kt_merged_product_20250929
    컬럼 정보:
    - 컬럼명: display_category_major
    - 데이터 타입: object
    - NULL 비율: 0.00%
    - 고유값 개수: 8
    - 기존 설명: 웹사이트에 노출되는 제품의 대분류 카테고리명입니다. (예: 모바일, TV & 오디오...

📋 응답 객체 타입: <class 'openai.types.chat.chat_completion.ChatCompletion'>
  - choices 개수: 1
  - choice 객체: Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. 짧은 설명  \n제품의 대분류 카테고리명\n\n2. 상세 설명  \n제품이 웹사이트에 노출될 때 적용되는 대분류 카테고리명입니다. (예: 모바일, 리빙가전, TV 등)  \n이 컬럼은 제품 그룹화를 통해 사용자가 카테고리별로 상품을 탐색할 수 있도록 하며, 8개의 고유 카테고리명을 현행 운영 중입니다.\n\n3. 데이터 특성  \n- NULL값 없음 (필수값)  \n- 사용 가능한 값: 8종(예: 모바일, 리빙가전, TV 등)  \n- 문자열 길이: 2~8자  \n- 주요 빈도: 모바일(813), 리빙가전(592), TV(430)  \n- 데이터 타입: 문자열(object)', refusal=None, role='assistant', annotations=[], audio=None, 

,column_name,data_type,null_ratio,unique_count,description,generated_at,min,max,mean,median
0,display_category_major,object,0.00%,8,1. 짧은 설명 \n제품의 대분류 카테고리명\n\n2. 상세 설명 \n제품이 웹...,2025-09-29T23:41:33.675733,NaN,NaN,NaN,NaN
1,display_category_middle,object,0.00%,41,1. 짧은 설명 \n제품의 웹 노출 중분류 카테고리명\n\n2. 상세 설명 \n...,2025-09-29T23:41:36.480025,NaN,NaN,NaN,NaN
2,display_category_minor,object,0.00%,183,"1. 짧은 설명 (한 줄, 20자 이내) \n제품 소분류 카테고리명\n\n2. 상...",2025-09-29T23:41:38.921735,NaN,NaN,NaN,NaN
3,product_category_major,object,0.00%,20,1. 짧은 설명 \n제품 대분류 카테고리 정보\n\n2. 상세 설명 \n내부 시...,2025-09-29T23:41:42.091229,NaN,NaN,NaN,NaN
4,product_category_middle,object,0.00%,80,1. 짧은 설명 \n제품의 중분류 카테고리명 또는 코드\n\n2. 상세 설명 \...,2025-09-29T23:41:44.297080,NaN,NaN,NaN,NaN
5,product_category_minor,object,0.00%,200,1. 짧은 설명 \n제품 소분류 코드 또는 명칭\n\n2. 상세 설명 \npro...,2025-09-29T23:41:46.983594,NaN,NaN,NaN,NaN
6,model_name,object,2.78%,2120,"1. 짧은 설명 (한 줄, 20자 이내) \n제품의 공식 모델명\n\n2. 상세 ...",2025-09-29T23:41:49.734963,NaN,NaN,NaN,NaN
7,model_code,object,0.00%,2774,"### 1. 짧은 설명 (한 줄, 20자 이내) \n제품별 고유 모델 코드\n\n...",2025-09-29T23:41:52.308077,NaN,NaN,NaN,NaN
8,product_id,object,0.00%,2774,1. 짧은 설명 \n제품별 고유 식별자\n\n2. 상세 설명 \n각 제품의 고유...,2025-09-29T23:41:54.476451,NaN,NaN,NaN,NaN
9,product_name,object,0.00%,1681,1. 짧은 설명 \n상품의 최종 표기명입니다.\n\n2. 상세 설명 \n제품의 ...,2025-09-29T23:41:56.824213,NaN,NaN,NaN,NaN


In [13]:
# 9. 테이블 전체 설명 생성
def generate_table_description(table_name='test', table_stats=None, column_stats=None):
    """테이블 전체에 대한 종합적인 설명 생성"""
    
    if not openai_client:
        return None
    
    # 주요 컬럼 정보 추출
    key_columns = []
    if column_stats is not None and len(column_stats) > 0:
        key_columns = column_stats.head(10)['column_name'].tolist()
    
    prompt = f"""
    다음 데이터베이스 테이블에 대한 종합적인 설명을 한국어로 작성해주세요.
    
    테이블명: {table_name}
    
    테이블 통계:
    - 전체 행 수: {table_stats.get('total_rows', 'N/A') if table_stats else 'N/A'}
    - 테이블 크기: {table_stats.get('table_size', 'N/A') if table_stats else 'N/A'}
    - 전체 컬럼 수: {len(column_stats) if column_stats is not None else 'N/A'}
    
    주요 컬럼: {', '.join(key_columns)}
    
    다음 내용을 포함하여 설명해주세요:
    1. 테이블의 주요 목적과 역할 (2-3줄)
    2. 저장되는 데이터의 비즈니스 의미
    3. 다른 테이블과의 잠재적 연관 관계
    4. 데이터 활용 사례 (예: 리포트, 분석, API 등)
    """
    
    try:
        print("--------")
        response = openai_client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=[
                {"role": "system", "content": "당신은 데이터 아키텍트입니다. 테이블의 비즈니스 목적과 활용 방안을 명확하게 설명해주세요."},
                {"role": "user", "content": prompt}
            ],
            # temperature 파라미터 제거 (기본값 1 사용)
            max_completion_tokens=500
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"❌ 테이블 설명 생성 실패: {e}")
        return None

# 테이블 설명 생성
table_description = generate_table_description(
    table_name='kt_merged_product_20250929',
    table_stats=table_stats,
    column_stats=df_column_stats
)

if table_description:
    print("=" * 60)
    print("📋 kt_merged_product_20250929 테이블 설명")
    print("=" * 60)
    print(table_description)
    print("=" * 60)

--------
📋 kt_merged_product_20250929 테이블 설명
1. 테이블의 주요 목적과 역할  
kt_merged_product_20250929 테이블은 KT에서 운영하는 제품 정보를 범주별로 통합·관리하기 위한 마스터 테이블로써, 다양한 분류 체계와 제품 속성 정보를 한 곳에 집약하여 제품 관리 및 비즈니스 의사결정에 활용합니다.

2. 저장되는 데이터의 비즈니스 의미  
이 테이블에는 제품의 대·중·소분류(디스플레이 및 내부 카테고리), 모델 정보, 제품 ID 및 명칭 등 주요 제품 식별 및 분류 정보가 저장되며, 이는 제품의 구조화된 관리와 상품군별 성과 분석, 신제품 출시에 필요한 핵심 데이터로 사용됩니다.

3. 다른 테이블과의 잠재적 연관 관계  
본 테이블은 판매 데이터(예: 주문 테이블), 재고 관리 테이블, 가격 정책 테이블 및 마케팅 캠페인 테이블 등과 연결되어 각 제품의 판매 이력, 재고 상황, 가격 변동 및 프로모션 이력 등 부가 정보를 참조하여 종합적인 제품 관리가 가능하도록 설계될 수 있습니다.

4. 데이터 활용 사례  
- 제품 분류별 판매 현황 리포트 및 시장 분석 자료 작성  
- API를 통한 제품 정보 제공(상품 리스트, 카테고리별 조회 등)  
- 신제품 출시에 따른 분류 체계 자동화 및 업무 시스템 연계  
- 상품군별 프로모션 전략 수립 및 효과 분석  
- 통합 제품 마스터를 기반으로 내·외부 시스템 간 데이터 동기화  
이와 같이 kt_merged_product_20250929 테이블은 다양한 비즈니스 분석과 운영 프로세스의 근간이 되는 핵심 제품 정보 저장소 역할을 합니다.


In [14]:
# 10. 메타데이터 저장 (JSON, CSV 형식)
def save_metadata(df_metadata, table_description, table_name, output_dir='./metadata'):
    """생성된 메타데이터를 파일로 저장"""
    
    import os
    from datetime import datetime
    
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. CSV 형식으로 컬럼 메타데이터 저장
    if df_metadata is not None:
        csv_path = os.path.join(output_dir, f'{table_name}_columns_{timestamp}.csv')
        df_metadata.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"✅ 컬럼 메타데이터 CSV 저장: {csv_path}")
    
    # 2. JSON 형식으로 전체 메타데이터 저장
    metadata_json = {
        'table_name': table_name,
        'generated_at': datetime.now().isoformat(),
        'table_description': table_description,
        'table_statistics': table_stats,
        'columns': df_metadata.to_dict('records') if df_metadata is not None else []
    }
    
    json_path = os.path.join(output_dir, f'{table_name}_metadata_{timestamp}.json')
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(metadata_json, f, ensure_ascii=False, indent=2, default=str)
    print(f"✅ 전체 메타데이터 JSON 저장: {json_path}")
    
    # 3. 테이블 설명만 별도 텍스트 파일로 저장
    if table_description:
        txt_path = os.path.join(output_dir, f'{table_name}_description_{timestamp}.txt')
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(f"{table_name} 테이블 설명\n")
            f.write("=" * 60 + "\n")
            f.write(f"생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 60 + "\n\n")
            f.write(table_description)
        print(f"✅ 테이블 설명 TXT 저장: {txt_path}")
    
    return {
        'csv_path': csv_path if df_metadata is not None else None,
        'json_path': json_path,
        'txt_path': txt_path if table_description else None
    }

# 메타데이터 저장
saved_files = save_metadata(df_metadata, table_description, table_name='kt_merged_product_20250929')
print("\n📁 저장된 파일:")
for key, path in saved_files.items():
    if path:
        print(f"  - {key}: {path}")

✅ 컬럼 메타데이터 CSV 저장: ./metadata/kt_merged_product_20250929_columns_20250929_234428.csv
✅ 전체 메타데이터 JSON 저장: ./metadata/kt_merged_product_20250929_metadata_20250929_234428.json
✅ 테이블 설명 TXT 저장: ./metadata/kt_merged_product_20250929_description_20250929_234428.txt

📁 저장된 파일:
  - csv_path: ./metadata/kt_merged_product_20250929_columns_20250929_234428.csv
  - json_path: ./metadata/kt_merged_product_20250929_metadata_20250929_234428.json
  - txt_path: ./metadata/kt_merged_product_20250929_description_20250929_234428.txt


In [15]:
# 11. 메타데이터 데이터베이스 저장 (선택사항)
def save_metadata_to_db(df_metadata, table_description, target_table='table_metadata', table_name='test'):
    """생성된 메타데이터를 데이터베이스에 저장"""
    
    conn = get_db_connection()
    if not conn:
        return False
    
    try:
        cursor = conn.cursor()
        
        # 메타데이터 테이블 생성 (없는 경우)
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {target_table} (
            id SERIAL PRIMARY KEY,
            table_name VARCHAR(255),
            column_name VARCHAR(255),
            data_type VARCHAR(100),
            null_ratio VARCHAR(20),
            unique_count INTEGER,
            min_value TEXT,
            max_value TEXT,
            mean_value NUMERIC,
            median_value NUMERIC,
            description TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_table_query)
        
        # 테이블 설명 저장용 테이블
        create_desc_table_query = """
        CREATE TABLE IF NOT EXISTS table_descriptions (
            id SERIAL PRIMARY KEY,
            table_name VARCHAR(255) UNIQUE,
            description TEXT,
            total_rows BIGINT,
            table_size VARCHAR(50),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(create_desc_table_query)
        
        # 기존 데이터 삭제 (중복 방지)
        cursor.execute(f"DELETE FROM {target_table} WHERE table_name = '{table_name}'")
        
        # 컬럼 메타데이터 삽입
        if df_metadata is not None:
            for _, row in df_metadata.iterrows():
                insert_query = f"""
                INSERT INTO {target_table} 
                (table_name, column_name, data_type, null_ratio, unique_count, 
                 min_value, max_value, mean_value, median_value, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (
                    table_name,
                    row.get('column_name'),
                    row.get('data_type'),
                    row.get('null_ratio'),
                    row.get('unique_count'),
                    str(row.get('min')) if 'min' in row else None,
                    str(row.get('max')) if 'max' in row else None,
                    row.get('mean') if 'mean' in row else None,
                    row.get('median') if 'median' in row else None,
                    row.get('description')
                ))
        
        # 테이블 설명 저장/업데이트
        if table_description and table_stats:
            upsert_query = """
            INSERT INTO table_descriptions (table_name, description, total_rows, table_size)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (table_name) 
            DO UPDATE SET 
                description = EXCLUDED.description,
                total_rows = EXCLUDED.total_rows,
                table_size = EXCLUDED.table_size,
                updated_at = CURRENT_TIMESTAMP
            """
            cursor.execute(upsert_query, (
                table_name,
                table_description,
                table_stats.get('total_rows'),
                table_stats.get('table_size')
            ))
        
        conn.commit()
        print(f"✅ 메타데이터가 데이터베이스에 저장되었습니다.")
        return True
        
    except Exception as e:
        print(f"❌ 데이터베이스 저장 실패: {e}")
        conn.rollback()
        return False
        
    finally:
        cursor.close()
        conn.close()

# 데이터베이스에 저장
if df_metadata is not None:
    save_metadata_to_db(df_metadata, table_description)

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 메타데이터가 데이터베이스에 저장되었습니다.


## 요약

이 노트북은 `sr_merged_product` 테이블에 대한 메타데이터를 생성하고 저장합니다.

### 수행된 작업:
1. ✅ PostgreSQL 연결 설정
2. ✅ Azure OpenAI 클라이언트 구성
3. ✅ 테이블 스키마 정보 조회
4. ✅ 테이블 통계 정보 수집 (행 수, 크기 등)
5. ✅ 컬럼별 상세 통계 분석
6. ✅ Azure OpenAI를 활용한 컬럼 설명 생성
7. ✅ 테이블 전체 설명 생성
8. ✅ 메타데이터 파일 저장 (CSV, JSON, TXT)
9. ✅ 데이터베이스에 메타데이터 저장

### 다음 단계:
- 전체 컬럼에 대한 메타데이터 생성 (현재는 10개만 처리)
- 데이터 품질 검증 규칙 추가
- 비즈니스 용어 사전과 연계
- 다른 테이블에 대한 메타데이터 생성